# Laboratorio 9
## Ataque y defensa de modelos de Deep Learning
Universidad del Valle de Guatemala<br>
Security Data Science<br>
Pablo Andrés Zamora Vásquez - 21780<br>
Diego Andrés Morales Aquino - 21762<br>

## Primera parte: Ataques

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from sklearn.metrics import accuracy_score
import numpy as np

In [13]:
# Modelo entrenado
model = keras.models.load_model("malware_classification_model.keras")

# Recompilar con eager mode habilitado
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True  # Necesario para ART con TF2
)

In [15]:
# Cargar datos
dataset_path = "malimg_paper_dataset_imgs/malimg_paper_dataset_imgs"
img_height, img_width = 64, 64
batch_size = 32
seed = 42

datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.3)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=seed
)

X_test, y_test = next(val_generator)
for _ in range(len(val_generator) - 1):
    x, y = next(val_generator)
    X_test = np.concatenate((X_test, x))
    y_test = np.concatenate((y_test, y))

Found 2790 images belonging to 25 classes.


In [16]:
# Envolver modelo con ART

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

### Ataque #1: FGSM (Evasión)

In [17]:
# Ataque FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.1)
X_adv = attack.generate(x=X_test)

In [18]:
# Evaluación
preds_original = np.argmax(classifier.predict(X_test), axis=1)
preds_adv = np.argmax(classifier.predict(X_adv), axis=1)
true_labels = np.argmax(y_test, axis=1)

print("Accuracy original:", accuracy_score(true_labels, preds_original))
print("Accuracy con ataque FGSM:", accuracy_score(true_labels, preds_adv))

Accuracy original: 0.9594982078853047
Accuracy con ataque FGSM: 0.22903225806451613


Este primer ataque consiste en generar ejemplos engañosos utilizando los gradientes de la red neuronal. Para ello, se calculan los gradientes de la función de pérdida con respecto a la imagen de entrada, y se utiliza esta información para crear una nueva imagen que maximice la pérdida. El objetivo es lograr que el modelo clasifique erróneamente estas entradas alteradas.

El parámetro eps de 0.1 (magnitud de perturbación) indica que se realizan cambios sutiles pero suficientes para confundir al modelo.

Al evaluar las imágenes modificadas para maximizar la pérdida del modelo, se puede observar que la precisión disminuye considerablemente con respecto a los datos originales. Esto evidencia la vulnerabilidad del modelo frente a perturbaciones diseñadas específicamente para engañarlo.

### Ataque #2: Black-box (Inferencia)

In [19]:
# Reutilizar el mismo ImageDataGenerator pero con subset="training"
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=False,
    seed=seed
)

# Convertir train_generator en arrays
X_train, y_train = next(train_generator)
for _ in range(len(train_generator) - 1):
    x, y = next(train_generator)
    X_train = np.concatenate((X_train, x))
    y_train = np.concatenate((y_train, y))


Found 6549 images belonging to 25 classes.


In [20]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
import numpy as np

# 1) Balancear tamaños
min_len = min(len(X_train), len(X_test))
X_train_bal, y_train_bal = X_train[:min_len], y_train[:min_len]
X_test_bal,  y_test_bal  = X_test[:min_len],  y_test[:min_len]

# 2) Instanciar el ataque
mi_attack = MembershipInferenceBlackBox(
    estimator=classifier,
    input_type="prediction",      # usa directamente las probabilidades del modelo
    attack_model_type="nn",
    scaler_type="minmax",         # normaliza las features antes de entrenar el adversario
    nn_model_epochs=50,
    nn_model_batch_size=32
)

# 3) Entrenar el ataque
mi_attack.fit(
    x=X_train_bal,
    y=y_train_bal,
    test_x=X_test_bal,
    test_y=y_test_bal
)

# 4) Inferir membership (pasando siempre las etiquetas)
pred_train = mi_attack.infer(x=X_train_bal, y=y_train_bal)
pred_test  = mi_attack.infer(x=X_test_bal,  y=y_test_bal)

# 5) Métricas
tpr       = np.mean(pred_train == 1)
fpr       = np.mean(pred_test  == 1)
advantage = tpr - fpr

print(f"TPR (miembros):    {tpr:.3f}")
print(f"FPR (no-miembros): {fpr:.3f}")
print(f"Advantage:      {advantage:.3f}")


TPR (miembros):    1.000
FPR (no-miembros): 0.513
Advantage:      0.487


Como segundo tipo, se tiene un ataque de inferencia de membresía tipo black box. Este busca determinar si ciertas imágenes fueron utilizadas durante el entrenemiento del modelo. Para el ataque se utiliza "MembershipInferenceBlackBox", indicándole que solo se debe tener acceso a las probabilidades de predicción y no a los datos internos. Al entrenar el modelo, se le pasan tanto ejemplos positivos (los datos que el modelo sí vio durante el entrenamiento) y datos negativos (imágenes que el modelo no vio). Al ejecutar la función infer sobre el modelo atacante, este intenta adivinar si cada imagen pertenece o no al conjunto de entrenamiento original. Los ejemplos positivos deberían de dar valores cercanos a 1, indicando que fueron identificados como miembros. Mientras que, los ejemplos negativos deberían ser cercanos a 0, indicando que no son miembros.

La métrica TPR representan la proporción de imágenes que sí pertenecen al conjunto de entrenamiento (miembros) correctamente identificadas. Mientras que, FPR es la proporción de no miembros incorrectamente clasificados como miembros. Advantage mide qué tanto mejor que al azar lo está haciendo el atacante. En este caso, el atacante identificó correctamente el 100% de los ejemplos que sí estaban en el conjunto de entrenamiento. Así como, clasificó erróneamente como miembros a más del 51% de los ejemplos que no estaban en el entrenamiento. Sin embargo, el advantage de 0.487 indica que se puede predecir la pertenencia al conjunto de entrenamiento casi un 49% mejor que el azar, lo cual refleja una fuga de información.



## Segunda parte: Defensas

### Defensa #1: Adversarial Training

Se reentrena el modelo incluyendo ejemplos FGSM para que aprenda a ser robusto

In [21]:
# Generar adversariales sobre tu train set
fgsm = FastGradientMethod(estimator=classifier, eps=0.1)
X_train_adv = fgsm.generate(x=X_train)
y_train_adv = y_train.copy()

#Combinar datos limpios + adversariales
X_comb = np.concatenate([X_train, X_train_adv], axis=0)
y_comb = np.concatenate([y_train, y_train_adv], axis=0)

# Clonar y recompilar tu modelo base
model_def = keras.models.clone_model(model)
model_def.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True
)

# Reentrenar sobre el conjunto combinado
model_def.fit(
    X_comb, y_comb,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

# Envolver el modelo defendido
classifier_def = TensorFlowV2Classifier(
    model=model_def,
    nb_classes=25,
    input_shape=(64,64,3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0,1.0)
)

# Evaluar defensa ante FGSM
X_adv_test = fgsm.generate(x=X_test)
acc_before = np.mean(np.argmax(model.predict(X_adv_test),axis=1) == np.argmax(y_test,axis=1))
acc_after  = np.mean(np.argmax(model_def.predict(X_adv_test),axis=1) == np.argmax(y_test,axis=1))
print(f"Accuracy FGSM antes de defensa: {acc_before:.3f}")
print(f"Accuracy FGSM tras adversarial training: {acc_after:.3f}")

Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 129s 392ms/step - accuracy: 0.4598 - loss: 1.8181 - val_accuracy: 0.4416 - val_loss: 1.5231
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 142s 393ms/step - accuracy: 0.7874 - loss: 0.6512 - val_accuracy: 0.5599 - val_loss: 1.5223
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 125s 381ms/step - accuracy: 0.8975 - loss: 0.3616 - val_accuracy: 0.5408 - val_loss: 2.1487
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 129s 393ms/step - accuracy: 0.9298 - loss: 0.2376 - val_accuracy: 0.5920 - val_loss: 1.6056
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 144s 399ms/step - accuracy: 0.9421 - loss: 0.1752 - val_accuracy: 0.5798 - val_loss: 1.9356
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 141s 395ms/step - accuracy: 0.9524 - loss: 0.1428 - val_accuracy: 0.5756 - val_loss: 2.4433
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 137s 381ms/step - accuracy: 0.9567 - loss: 0.1335 - val_accuracy: 0.5798 - val_loss: 1.9578
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 145s 391ms/step - accuracy: 0.9657 -

Esta defensa consiste en fortalecer el modelo al exponerlo a ejemplos adversariales durante su entrenamiento. Para ello primero se generan imágenes perturbadas del conjunto de entrenamiento, combinándolo con los datos originales para formar un nuevo conjunto de entrenamiento más robusto.

Bajo ataque FGSM el modelo original solo acertaba el 23.2 % de las imágenes adversariales. Tras reentrenar con ejemplos FGSM, el mismo adversario logra solo un 83.5 % de acierto, recuperando robustez y subiendo la accuracy en adversariales desde 0.23 hasta 0.84. Esto significa que adversarial training es muy efectivo atacando directamente la perturbación FGSM, pues el modelo aprendió a reconocer y resistir esas pequeñas modificaciones.

### Defensa 2: Feature Squeezing

Aplicar un preprocesamiento que reduce la cantidad de “bits” de información de cada píxel, haciendo más difícil ocultar perturbaciones pequeñas.

In [22]:
from art.defences.preprocessor import FeatureSqueezing

# 1) Crear el preprocesador
squeezer = FeatureSqueezing(
    clip_values=(0.0, 1.0),
    bit_depth=2
)

# 2) Aplicar el preprocesador a los adversariales
X_squeezed, _ = squeezer(X_adv_test, y=y_test)

# 3) Evaluar
acc_clean    = np.mean(
    np.argmax(model.predict(X_squeezed),    axis=1)
    == np.argmax(y_test, axis=1)
)
acc_defended = np.mean(
    np.argmax(model_def.predict(X_squeezed), axis=1)
    == np.argmax(y_test, axis=1)
)

print(f"Accuracy con Feature Squeezing (modelo original): {acc_clean:.3f}")
print(f"Accuracy con Feature Squeezing (modelo defendido): {acc_defended:.3f}")

88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step
Accuracy con Feature Squeezing (modelo original): 0.374
Accuracy con Feature Squeezing (modelo defendido): 0.935


En esta defensa se está reduciendo la precisión de las características de entrada para hacer más dificil que ataques adversariales engañen al modelo.

Aplicando el preprocesador de bit-depth=2 sobre el modelo sin retraining, la accuracy ante FGSM sube del 23.2% a 38.3%. Si se combina el squeezing con el modelo adversarialmente entrenado, la accuracy salta hasta 93.5 % sobre esos mismos ejemplos adversariales. Esto indica que Feature Squeezing, al reducir la información por píxel, ya aporta cierta protección al modelo original y refuerza aún más al modelo entrenado con adversariales.

### Defensa de ataque 2: FeatureSqueezing y JpegCompression

In [51]:
from art.defences.preprocessor import FeatureSqueezing, JpegCompression
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox

jpeg_compressor = JpegCompression(clip_values=(0.0, 1.0), quality=75)
fsq = FeatureSqueezing(clip_values=(0.0, 1.0), bit_depth=10)

# Aplicar ambos preprocesamientos secuencialmente a los datos
X_train_def, _ = jpeg_compressor(X_train)
X_train_def, _ = fsq(X_train_def)

X_test_def, _ = jpeg_compressor(X_test)
X_test_def, _ = fsq(X_test_def)

# Clonar y recompilar modelo
model_def = keras.models.clone_model(model)
model_def.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True
)

# Reentrenar el modelo con datos defendidos
model_def.fit(
    X_train_def, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

# Envolver el modelo con ART
classifier_def = TensorFlowV2Classifier(
    model=model_def,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

# Ataque de inferencia de membresía
mi_attack_def = MembershipInferenceBlackBox(
    estimator=classifier_def,
    input_type="prediction",
    attack_model_type="nn",
    scaler_type="minmax",
    nn_model_epochs=50,
    nn_model_batch_size=32
)

# Entrenar al atacante con datos defendidos
mi_attack_def.fit(x=X_train_def, y=y_train, test_x=X_test_def, test_y=y_test)

# Inferencia del atacante
pred_train_def = mi_attack_def.infer(x=X_train_def, y=y_train)
pred_test_def  = mi_attack_def.infer(x=X_test_def,  y=y_test)

tpr_def = np.mean(pred_train_def == 1)
fpr_def = np.mean(pred_test_def == 1)
advantage_def = tpr_def - fpr_def

print("\n--- Con JpegCompression + FeatureSqueezing ---")
print(f"TPR (miembros):    {tpr_def:.3f}")
print(f"FPR (no-miembros): {fpr_def:.3f}")
print(f"Advantage:         {advantage_def:.3f}")


Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 57s 395ms/step - accuracy: 0.4162 - loss: 1.8523 - val_accuracy: 0.0236 - val_loss: 10.8635
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 56s 393ms/step - accuracy: 0.7404 - loss: 0.6225 - val_accuracy: 0.0253 - val_loss: 16.5526
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 81s 384ms/step - accuracy: 0.8711 - loss: 0.3434 - val_accuracy: 0.0253 - val_loss: 20.8891
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 83s 390ms/step - accuracy: 0.9535 - loss: 0.1425 - val_accuracy: 0.0253 - val_loss: 22.2467
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 57s 398ms/step - accuracy: 0.9423 - loss: 0.1733 - val_accuracy: 0.0253 - val_loss: 25.9724
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 54s 377ms/step - accuracy: 0.9744 - loss: 0.0940 - val_accuracy: 0.0253 - val_loss: 23.0743
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 57s 393ms/step - accuracy: 0.9789 - loss: 0.0644 - val_accuracy: 0.0253 - val_loss: 26.1945
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 83s 399ms/step - accuracy: 0.9709 - 

La defensa del modelo consiste en aplicar técnicas de preprocesamiento, como la compresión JPEG y el "Feature Squeezing", para proteger el modelo de ataques de inferencia de membresía. En primer lugar, se preprocesan los datos de entrenamiento y test utilizando compresión JPEG para reducir la calidad de la imagen, mientras que se usa "Feature Squeezing" para reducir la profundidad de bits de las características.

Los resultados con la defensa muestran una reducción en la efectividad del ataque. Aunque el TPR disminuye ligeramente de 1.000 a 0.984, el FPR aumenta considerablemente de 0.513 a 0.861, lo que indica que el atacante comete más falsos positivos. Esto provoca una disminución en el Advantage, que pasa de 0.487 sin defensa a 0.123 con defensa. Aunque esta mejora sugiere que el modelo tiene un mayor grado de protección, no se lograron resultados óptimos, ya que aún es capaz de identificar correctamente una gran cantidad de imágenes. Además de la combinación de defensas, se probaron ambas técnicas de manera individual, obteniendo resultados algo inferiores. Por otro lado, se intentó aplicar LabelSmoothing, pero este enfoque favoreció al ataque, permitiendo que se identificara correctamente al 100% de los miembros sin generar falsos positivos.